In [49]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.statespace.varmax import VARMAX

In [50]:
from read_from_db import read_from_db

df_main, df_time = read_from_db()

In [51]:
df = df_main[[ 'time_stamp', 'latitude', 'longitude', 'temperature']].head(1000)


In [52]:
df_2 = df.merge(df_time, how='left',on='time_stamp')
df_3 = df_2.groupby(by=['month', 'day']).mean()


C:\Users\Stefan\AppData\Local\Temp\ipykernel_22184\3529372596.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [53]:
data = df_3[['latitude','longitude', 'temperature']]

In [54]:
train_data = data[:-5]
test_data = data[-5:]
print(train_data)
print(test_data)

            latitude  longitude  temperature
month day                                   
8     13  -24.785744  31.888306    27.769231
      14  -24.736101  31.905263    28.200000
      15  -24.733130  31.912098    29.166667
      16  -24.755686  31.891327    24.347826
      17  -24.758422  31.889737    25.583333
      18  -24.760871  31.895547    26.600000
      19  -24.753416  31.894981    25.000000
      20  -24.734299  31.908068    25.428571
      21  -24.732315  31.914961    26.909091
      22  -24.749825  31.929712    26.818182
      23  -24.753992  31.929588    28.266667
      24  -24.763345  31.928895    26.266667
      25  -24.754262  31.929976    23.411765
      26  -24.745417  31.926028    27.000000
      27  -24.672792  31.896388    27.250000
      28  -24.545190  31.868156    26.500000
      29  -24.486969  31.874171    29.294118
      30  -24.475361  31.913404    28.950000
      31  -24.379953  31.910108    31.181818
9     1   -24.332524  31.891010    28.888889
      2   

In [55]:
# fit model

model = VARMAX(train_data, order=(1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.forecast(steps=5)
#varmaxresults.apply
print(yhat)


c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning:

Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.

c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



     latitude  longitude  temperature
52 -24.556323  31.843857    27.682413
53 -24.556602  31.859801    27.760373
54 -24.562410  31.871702    27.590089
55 -24.569358  31.878819    27.598528
56 -24.574589  31.882569    27.637441


c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



In [56]:

fig = px.line(data, y='latitude', x='longitude', markers=True)
fig.add_trace(go.Scatter(y= yhat['latitude'], x=yhat['longitude']))
fig.add_trace(go.Scatter(y= test_data['latitude'], x=test_data['longitude']))
fig.show()

In [ ]:
fig = px.line_mapbox(data, lat="latitude", lon="longitude",  zoom=7, height=300)
fig.add_trace(px.line_mapbox(yhat,lat='latitude', lon='longitude').data[0])
fig.update_traces(line=dict(color='purple'), selector=0)
fig.update_traces(line=dict(color='red'), selector=1)
fig.update_layout(mapbox_style="stamen-terrain")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [57]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = data['longitude'],
    lat = data['latitude'],
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = yhat['longitude'],
    lat = yhat['latitude'],
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = test_data['longitude'],
    lat = test_data['latitude'],
    marker = {'size': 10}))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': data['longitude'].mean(), 'lat': data['latitude'].mean()},
        'style': "stamen-terrain",
        'zoom': 9})

fig.show()

In [4]:
df_merged = pd.merge(df_main, df_time, how='left', on='time_stamp')
df_merged = df_merged[['latitude', 'longitude', 'temperature', 'tag_id', 'year', 'month', 'day']]
df_daily_avg = df_merged.groupby(by=['tag_id', 'year', 'month', 'day']).mean()

In [40]:
df_test = df_daily_avg.loc['AM105']
model = VARMAX(df_test, order=(1, 1))
model_fit = model.fit(disp=False)

c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning:

Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.

c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [43]:
df_test_2 = df_daily_avg.loc['AM107']
model_2 = VARMAX(df_test_2, order=(3,2))
model_2_fit = model.fit(disp=False)

c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning:

Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.

c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

c:\Users\Stefan\Documents\Modul11\pink_elephants\venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [36]:
print(df_test.head(16))
prediction

                 latitude  longitude  temperature
year month day                                   
2007 8     13  -24.985765  31.273914    27.600000
           14  -24.986737  31.274279    26.219512
           15  -24.987447  31.275916    27.279070
           16  -24.991337  31.288865    26.043478
           17  -24.988214  31.300527    25.458333
           18  -24.987085  31.281499    24.487805
           19  -24.982304  31.266609    30.642857
           20  -24.982566  31.265410    25.170213
           21  -24.984751  31.269743    24.720930
           22  -24.986298  31.274475    25.285714
           23  -24.986338  31.276345    27.384615
           24  -24.989733  31.286811    26.125000
           25  -24.976476  31.312640    22.648649
           26  -24.970522  31.327548    26.042553
           27  -24.973160  31.331857    28.804878
           28  -24.976587  31.325613    26.416667


latitude  longitude  temperature
year month day                                   
2007 8     23  -24.761836  31.928651    27.742820
           24  -24.755354  31.924019    28.245281
           25  -24.770922  31.926339    27.445567
           26  -24.752310  31.927723    25.995978
           27  -24.746514  31.920907    27.466234
           28  -24.645956  31.885197    27.402672

In [45]:
df_test_2.tail(10)
print(type(model_fit))
#varmaxresults.apply
prediction = model_2_fit.predict(10,15)
true_val = df_test_2.head(15)

<class 'statsmodels.tsa.statespace.varmax.VARMAXResultsWrapper'>


In [46]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = true_val['longitude'],
    lat = true_val['latitude'],
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = prediction['longitude'],
    lat = prediction['latitude'],
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = true_val.iloc[10:15]['longitude'],
    lat =  true_val.iloc[10:15]['latitude'],
    marker = {'size': 10}))


fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': true_val['longitude'].mean(), 'lat': true_val['latitude'].mean()},
        'style': "stamen-terrain",
        'zoom': 9})

fig.show()

In [ ]:
df_merged = pd.merge(df_main, df_time, how='left', on='time_stamp')
df_merged = df_merged[['latitude', 'longitude', 'temperature', 'tag_id', 'year', 'month', 'day']]
df_daily_avg = df_merged.groupby(by=['tag_id', 'year', 'month', 'day']).mean()
df_daily_avg.reset_index(inplace=True)

elephant_tags = df_daily_avg['tag_id'].drop_duplicates().reset_index(drop=True)
print(elephant_tags)

In [ ]:
for tag in elephant_tags:    
    data = df_daily_avg[df_daily_avg['tag_id'] == tag][['latitude', 'longitude', 'temperature']]
    model = VARMAX(data, order = (1,1))
    model_fit = model.fit(disp = False)
    model_fit.save(f'./data/ML/varma_1_1_{tag}')

In [ ]:

loaded_model = VARMAXResults.load('./data/ML/varma_1_1_AM105')
#print(loaded_model.summary())
print(loaded_model.predict(100,110))
print(df_daily_avg.iloc[100:110])

In [ ]:
i = 10
pred = loaded_model.predict(i, i+10)
true_val = df_daily_avg.iloc[i:i+11]
fig = go.Figure(
    # go.Scattermapbox(
    # mode = "markers+lines",
    # lon = true_val['longitude'],
    # lat = true_val['latitude'],
    # marker = {'size': 10})
)

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = pred['longitude'],
    lat = pred['latitude'],
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = true_val['longitude'],
    lat =  true_val['latitude'],
    marker = {'size': 10}))


fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': pred['longitude'].mean(), 'lat': pred['latitude'].mean()},
        'style': "stamen-terrain",
        'zoom': 8})

fig.show()

In [ ]:
# only first elephant, forecasts
df_forecast = df_daily_avg[df_daily_avg['tag_id'] == 'AM105'][['latitude','longitude', 'temperature']].iloc[:240]


train_data = df_forecast[:-5]
test_data = df_forecast[-5:]

# fit model
model = VARMAX(train_data, order=(1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.forecast(steps=len(test_data))
print(yhat)

In [ ]:
model_fit.save('./data/ML/varma_forecast_1')

In [ ]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = df_forecast['longitude'].iloc[225:236],
    lat = df_forecast['latitude'].iloc[225:236],
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = yhat['longitude'],
    lat = yhat['latitude'],
    marker = {'size': 10}))

fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lon = test_data['longitude'],
    lat = test_data['latitude'],
    marker = {'size': 10}))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': yhat['longitude'].mean(), 'lat': yhat['latitude'].mean()},
        'style': "stamen-terrain",
        'zoom': 12})

fig.show()